In [1]:
# pip install pytesseract pdf2image opencv-python poppler

In [2]:
import os
import tempfile
import re

import cv2 as cv2
import pytesseract
import json
import numpy as np
import tabula

from pdf2image import convert_from_path
from PIL import Image

In [3]:
# Initial paths
poppler_path='C://Users//pirun//Downloads//Compressed//poppler-0.68.0_x86//poppler-0.68.0//bin'
pdf_path='C:/Users/pirun/OCR/OCR Task/PDF'

In [ ]:
extractedInfo=[]
pageInfo={}

In [4]:
# convet pdfs and save it in a folder it can handle mul
def ConvertImages(pdf_path,file):
    f=os.path.join(pdf_path,file)
    pages = convert_from_path(f,500,poppler_path=poppler_path)
    
    parent_dir_img= pdf_path.rsplit('/',1)[0]+'/Image/'
    pdf_image_path=parent_dir_img
    if (not os.path.exists(parent_dir_img)):
        os.mkdir(parent_dir_img)
        
    child_dir = pdf_path.rsplit('/',1)[0]+'/Image/'+file.rsplit('.',1)[0]+'/'
    if (not os.path.exists(child_dir)):
        os.mkdir(child_dir)
       
    page_counter =1
    for page in pages:
        filename = "page_"+str(page_counter)+".jpg"
        saveLocation = os.path.join(child_dir,filename)
        page.save(saveLocation,"JPEG")
        
        # parse the doccument
        GetHeaderItems(page_counter , saveLocation)
        
        page_counter=page_counter+1 

In [5]:
def GetHeaderItems(page_counter , file_directory):
    pageInfo={}
    headerDetails = []
   
    pageInfo['Page_Number'] = page_counter
    image = Image.open(file_directory)
    
    RegionTopLeft(image,headerDetails)
    
    RegionBottomLeftFooter(image,headerDetails)
    
    RegionTopRight(image,headerDetails)
    
    RegionFull(image,headerDetails)
    
    NameRegion(image,headerDetails)
    
    RegionBody(image,pageInfo)

    pageInfo["Key_Values"] = headerDetails
    
    extractedInfo.append(pageInfo)

    
    
    
def RegionTopLeft(image,headerDetails):
    
    width,height = image.size
    cropString=(0,0,width/2,height/3)
    image = image.crop(cropString)
    
    text = pytesseract.image_to_string(image, config = '--psm 6')
    
    patientName = re.search('(Patient)\s*?((Name|)\s*):?\s(.*)', text , re.IGNORECASE)
    if(patientName):
         headerDetails.append(patientName.group())

    gstRegNo = re.search(r'GST REG NO\s*:?\s*(.*)', text , re.IGNORECASE)
    if(gstRegNo):
        headerDetails.append(gstRegNo.group())


        
def RegionBottomLeftFooter(image,headerDetails):
    
    width,height = image.size
    cropString=(0,height/10*9,width/3,height)
    image = image.crop(cropString)
    
    text = pytesseract.image_to_string(image, config = '--psm 6')
    
    pageOf= re.search('(Page)\s*(.*)', text , re.IGNORECASE)
    if(pageOf):
        headerDetails.append("Page : " + str(pageOf.group()))

#     hospitalClinic = re.search('(.*)(Hos(.*)|cli(.*))(.*)', text , re.IGNORECASE)
#     if(hospitalClinic and not nameAdded):
#         headerDetails.append(hospitalClinic.group())



def RegionTopRight(image,headerDetails):
    
    width,height = image.size
    cropString=(width/2,0,width,height/3)
    image = image.crop(cropString)
    
    text = pytesseract.image_to_string(image, config = '--psm 6')

    pageOf= re.search('(Page)\s*(.*)',text , re.IGNORECASE)
    if(pageOf):
        headerDetails.append("Page : " + str(pageOf.group()))

    gstRegNo = re.search(r'GST REG NO\s*(.*)', text , re.IGNORECASE)
    if(gstRegNo):
        headerDetails.append(gstRegNo.group())

    patientNationalID = re.search(r'\b(.*?NRIC(.*))\b', text , re.IGNORECASE)
    if(patientNationalID):
        headerDetails.append(patientNationalID.group())

    visitDate = re.search(r'(Visit)\s*(Date)\s*(.*)', text , re.IGNORECASE)
    if(visitDate):
        headerDetails.append(visitDate.group())

    taxInvoiceDate = re.search(r'(Tax\s*)?(Invoice)\s*(Date)\s*(.*)', text , re.IGNORECASE)
    if(taxInvoiceDate):
        headerDetails.append(taxInvoiceDate.group())

    BillReceiptDate = re.search(r'((Bill\s*)|(Receipt\s*))(Date)\s*(.*)', text , re.IGNORECASE)
    if(BillReceiptDate):
        headerDetails.append(BillReceiptDate.group())

    admissionDate = re.search(r'(Admission)\s*(Date)\s*(.*)', text , re.IGNORECASE)
    if(admissionDate):
        headerDetails.append(admissionDate.group())

    billReferenceNumber = re.search(r'(((Bill\s*)?(Ref|Reference)\s*)|(case\s*))\s*(Number|No)\s*(.*)', text , re.IGNORECASE)
    if(billReferenceNumber):
        headerDetails.append(billReferenceNumber.group())

    location = re.search(r'(.*)?(Location)\s*(.*)', text , re.IGNORECASE)
    if(location):
        headerDetails.append(location.group())


        
def RegionFull(image,headerDetails):
    
    text = pytesseract.image_to_string(image, config = '--psm 6')

    billType = re.search('(ORIGINAL|DUPLICATE|INTERIM|Adjusted)', text , re.IGNORECASE)
    if(billType):
        headerDetails.append("BILL TYPE : " + str(billType.group()))
        
    dischargeDate = re.search('([0-3]\s)([0-9]\s)[JFMASOND][AEPUECO][NBRYLGPTVC]\s[2][0-9][0-9][0-9]', text , re.IGNORECASE)
    if(dischargeDate):
        headerDetails.append("Discharge Date : " + str(dischargeDate.group()))
              
    totalPayable = re.search('(Net|Total)\s(Amount)\s(.*)', text , re.IGNORECASE)
    if(totalPayable):
        headerDetails.append(totalPayable.group())
    
    doctorName = re.search(r'served by\s(.*)', text , re.IGNORECASE)
    if(doctorName):
        headerDetails.append(doctorName.group())
            
              

def NameRegion(image,headerDetails):
    
    width,height = image.size
    cropString=(width/11,0,width/3.6,height/10*1)
    image = image.crop(cropString)
    
    text = pytesseract.image_to_string(image, config = '--psm 6')

    hospitalClinic = re.search(r'(.*)(\r?\n)?(.*)(Hos(.*)|cli(.*))', text , re.IGNORECASE)
    if(hospitalClinic):
        headerDetails.append("Hospital : " + str(hospitalClinic.group().replace("\n" , " ")))
        
            

def RegionBody(image,pageInfo):
                             
    table=[]   
    
    width,height = image.size
    cropString=(0,height/4,width,height/4*3)
    image = image.crop(cropString)
                             
    text = pytesseract.image_to_string(image, config = '--psm 6')
                             
    lines = text.split('\n')
    
    for line in lines:
        
        tableContent={}
        
        match = re.search('(([A-Za-z]|[0-9]|[A-Za-z0-9]|[A-Z0-9]|[a-z0-9]){5,10})\s+(.+)([1-9]{1,3})\s+([0-9]{1,5})(\.|\,)([0-9]{2})',line , re.IGNORECASE)
        if(match):
            tableContent["CODE"]=str(match.group()).split(" ",1)[0]
            tableContent["DESCRIPTION"]=str(match.group()).rsplit(" ",2)[0].split(" ",1)[1]
            tableContent["QUANTITY"]=str(match.group()).rsplit(" ",2)[1]
            tableContent["AMMOUNT"]=str(match.group()).rsplit(" ",2)[2]
            table.append(tableContent)
    
    pageInfo["Table"] = table
    
    
    
def addItem(entry,headerDetails):
    
    headerDetails[(entry.group()).split(":",1)[0]]=(entry.group()).split(":",1)[1]
    
    

In [6]:
for file in os.listdir(pdf_path):
    f=os.path.join(pdf_path,file)
    if os.path.isfile(f):
        ConvertImages(pdf_path,file)
    
    print(extractedInfo)

[{'Page_Number': 1, 'Table': [{'CODE': 'HLA03074', 'DESCRIPTION': 'HLA DOB TYPING (PCR INT TO HIGH RES)', 'QUANTITY': '1', 'AMMOUNT': '243.93'}, {'CODE': 'HF703102', 'DESCRIPTION': 'HANDLING FEE', 'QUANTITY': '1', 'AMMOUNT': '18.00'}, {'CODE': 'HLA09027', 'DESCRIPTION': 'HLA ABCDR TYPING (SBT) FOR LOCAL TRANSPLANT', 'QUANTITY': '1', 'AMMOUNT': '885.98'}, {'CODE': 'HF703102', 'DESCRIPTION': 'HANDLING FEE', 'QUANTITY': '1', 'AMMOUNT': '18,00'}, {'CODE': '10009606', 'DESCRIPTION': 'CREATININE CLEARANCE TEST (CCT)', 'QUANTITY': '1', 'AMMOUNT': '24.90'}, {'CODE': '10010709', 'DESCRIPTION': 'GLUCOSE-6-PHOSPHATE DEHYDROGENAS', 'QUANTITY': '1', 'AMMOUNT': '23.30'}, {'CODE': '10010709', 'DESCRIPTION': 'GLUCOSE-6-PHOSPHATE DEHYDROGENAS', 'QUANTITY': '1', 'AMMOUNT': '23.30'}, {'CODE': '10019326', 'DESCRIPTION': 'VARICELLA-ZOSTER IGG AB', 'QUANTITY': '1', 'AMMOUNT': '43.80'}, {'CODE': '10024120', 'DESCRIPTION': 'CYTOMEGALOVIRUS IGM AB', 'QUANTITY': '1', 'AMMOUNT': '54.00'}, {'CODE': '10025806', 'D

In [7]:
for pages in extractedInfo:
    print("Page Number : " + str(pages["Page_Number"]))
    print("\n")
    
    for keyValues in pages["Key_Values"]:
        print(keyValues)
    print("\n") 
    
    for tableValues in pages["Table"]:
        print(tableValues)
    print("\n\n")
   

Page Number : 1


Patient : dasdasdasd
GST REG NO : M90368910N
Page : Page 1 of 2
Patient NRIC/HRN :S1s84154das
Visit Date *22,10.2018 1153 hrs
Tax Invoice Date  :22.10.2018 1546 hrs
Bill Date : 25.10.2018
Admission Date : 22.10.2018
Bill Ref Number -6818337074E-0010-01
Visit / Bill Location :GCLHAEC / GCLHAEC /
BILL TYPE : ORIGINAL
Discharge Date : 2 8 NOV 2018
NET AMOUNT PAYABLE IeCin vine 2457.28
served by ZAWIYAH BINTE ABUDULLAH ™*
Hospital : _ If Ccperal Hospital


{'CODE': 'HLA03074', 'DESCRIPTION': 'HLA DOB TYPING (PCR INT TO HIGH RES)', 'QUANTITY': '1', 'AMMOUNT': '243.93'}
{'CODE': 'HF703102', 'DESCRIPTION': 'HANDLING FEE', 'QUANTITY': '1', 'AMMOUNT': '18.00'}
{'CODE': 'HLA09027', 'DESCRIPTION': 'HLA ABCDR TYPING (SBT) FOR LOCAL TRANSPLANT', 'QUANTITY': '1', 'AMMOUNT': '885.98'}
{'CODE': 'HF703102', 'DESCRIPTION': 'HANDLING FEE', 'QUANTITY': '1', 'AMMOUNT': '18,00'}
{'CODE': '10009606', 'DESCRIPTION': 'CREATININE CLEARANCE TEST (CCT)', 'QUANTITY': '1', 'AMMOUNT': '24.90'}
{'CO